# Data Munging ReEDS output data files for input installations

To explore different scenarios for furture installation projections of PV (or any technology), ReEDS output data can be useful in providing standard scenarios. This input data will be used in the module files input to the PVDEMICE tool. Some will be used to explore middle, low and high projections, some for the Solar Futures Report. This journal extracts the data relevant for the current status of the PVDEMICE tool from ReEDS outputs.

In [1]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
import os
from pathlib import Path

reedsFile = str(Path().resolve().parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures.xlsx')
testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP')

print ("Input file is stored in %s" % reedsFile)
print ("Your simulation will be stored in %s" % testfolder)


Input file is stored in C:\Users\sayala\Documents\GitHub\December Core Scenarios ReEDS Outputs Solar Futures.xlsx
Your simulation will be stored in C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


In [3]:
cwd = os.getcwd() #grabs current working directory
rawdf = pd.read_excel(reedsFile,
                        sheet_name="Solar Capacity (GW)")
                        #index_col=[0,2,3]) #this casts scenario, PCA and State as levels
#now set year as an index in place
rawdf.drop(columns=['State'], inplace=True)
rawdf.set_index(['scenario','year','PCA'], inplace=True)


In [4]:
rawdf.index.get_level_values('scenario').unique()

Index(['Reference.Mod', 'Reference.Adv', 'Reference.Adv+DR', '95-by-35.Mod',
       '95-by-35.Adv', '95-by-35.Adv+DR', '95-by-35+Elec.Mod',
       '95-by-35+Elec.Adv', '95-by-35+Elec.Adv+DR'],
      dtype='object', name='scenario')

In [5]:
for ii in range (len(rawdf.unstack(level=1))):
    PCA = rawdf.unstack(level=1).iloc[ii].name[1]
    SCEN = rawdf.unstack(level=1).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = SCEN+'_'+PCA +'.csv'
    filetitle = os.path.join(testfolder, filetitle)
    A = rawdf.unstack(level=1).iloc[0]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = A.resample('Y').asfreq()
    A = A['new_Installed_Capacity_[MW]'].fillna(0).groupby(A['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')    
    A = pd.DataFrame(A)
    A.to_csv(filetitle)
    

In [6]:
# EXAMPLE FOR JUST ONE 
ii = 0
PCA = rawdf.unstack(level=1).iloc[ii].name[1]
SCEN = rawdf.unstack(level=1).iloc[ii].name[0]
SCEN=SCEN.replace('+', '_')
filetitle = SCEN+'_'+PCA +'.csv'
filetitle = os.path.join(testfolder, filetitle)
A = rawdf.unstack(level=1).iloc[0]
A = A.droplevel(level=0)
A.name = 'new_Installed_Capacity_[MW]'
A = pd.DataFrame(A)
A.index=pd.PeriodIndex(A.index, freq='A')
B = A.resample('Y').asfreq()
B = B['new_Installed_Capacity_[MW]'].fillna(0).groupby(B['new_Installed_Capacity_[MW]'].notna().cumsum()).transform('mean')
B = pd.DataFrame(B)
B.to_csv(filetitle)


## Playing with Multiindex Stuff

In [7]:
rawdf.unstack(level=0).head()
rawdf.unstack(level=1).head()
rawdf.unstack(level=2).head()


Capacity (GW)                                           \
PCA                               p1        p10      p100      p101      p102   
scenario          year                                                          
95-by-35+Elec.Adv 2010      0.000015   0.889023  0.000000  0.021315  0.014625   
                  2012      0.009394   1.459950  0.000000  0.061064  0.022944   
                  2014      0.022386   5.998475  0.000443  0.093015  0.032459   
                  2016      0.055574  10.657953  0.001196  0.386462  0.052036   
                  2018      0.090955  13.125968  0.015237  1.130471  0.411885   

                                                                          ...  \
PCA                         p103      p104      p105      p106      p107  ...   
scenario          year                                                    ...   
95-by-35+Elec.Adv 2010  0.000000  0.000000  0.000000  0.000000  0.000000  ...   
                  2012  0.000000  0.000000  0.007500  0.002000  0.000000  ...   
                  2014  0.011624  0.000895  0.016721  0.080478  0.002391  ...   
                  2016  0.029727  0.007580  0.050712  0.095206  0.030154  ...   
                  2018  0.132168  0.009026  0.101439  0.105268  0.080254  ...   

                                                                          \
PCA                          p90       p91       p92       p93       p94   
scenario          year                                                     
95-by-35+Elec.Adv 2010  0.000000  0.000395  0.000105  0.000000  0.000794   
                  2012  0.000000  0.002320  0.044345  0.000000  0.021545   
                  2014  0.000866  0.004385  0.094758  0.000196  0.096873   
                  2016  0.001711  0.008632  0.134988  0.000912  1.001218   
                  2018  0.106299  0.141674  0.248195  0.004965  1.057618   

                                                                          
PCA                          p95       p96       p97       p98       p99  
scenario          year                                                    
95-by-35+Elec.Adv 2010  0.000000  0.000000  0.007655  0.002195  0.000000  
                  2012  0.000000  0.000000  0.058629  0.106780  0.000000  
                  2014  0.000483  0.003359  0.159170  0.602453  0.006510  
                  2016  0.016740  0.048056  0.486106  2.006075  0.044939  
                  2018  0.072544  0.476227  0.816477  3.297150  0.330667  

[5 rows x 134 columns]

In [8]:
rawdf.unstack(level=1).iloc[0]


               year
Capacity (GW)  2010    0.000015
               2012    0.009394
               2014    0.022386
               2016    0.055574
               2018    0.090955
               2020    0.152445
               2022    0.156871
               2024    0.175173
               2026    1.702105
               2028    1.795453
               2030    1.988055
               2032    2.143502
               2034    2.259462
               2036    2.341862
               2038    2.409682
               2040    2.487484
               2042    2.573273
               2044    2.652801
               2046    2.683043
               2048    2.785752
               2050    2.890179
Name: (95-by-35+Elec.Adv, p1), dtype: float64

In [9]:
rawdf.unstack(level=1).iloc[2].name[1]

'p100'

In [10]:
rawdf.loc[('Reference.Mod',2010)].head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,Capacity (GW)
PCA,
p1,0.000015
p10,0.889023
p100,0.000000
p101,0.021315
p102,0.014625


In [11]:
scenarios = rawdf.groupby(level=0)
PCA = rawdf.groupby(level=2)

In [12]:
for a,b in scenarios:
    for c,d in PCA:
        print(a, c)

95-by-35+Elec.Adv p1
95-by-35+Elec.Adv p10
95-by-35+Elec.Adv p100
95-by-35+Elec.Adv p101
95-by-35+Elec.Adv p102
95-by-35+Elec.Adv p103
95-by-35+Elec.Adv p104
95-by-35+Elec.Adv p105
95-by-35+Elec.Adv p106
95-by-35+Elec.Adv p107
95-by-35+Elec.Adv p108
95-by-35+Elec.Adv p109
95-by-35+Elec.Adv p11
95-by-35+Elec.Adv p110
95-by-35+Elec.Adv p111
95-by-35+Elec.Adv p112
95-by-35+Elec.Adv p113
95-by-35+Elec.Adv p114
95-by-35+Elec.Adv p115
95-by-35+Elec.Adv p116
95-by-35+Elec.Adv p117
95-by-35+Elec.Adv p118
95-by-35+Elec.Adv p119
95-by-35+Elec.Adv p12
95-by-35+Elec.Adv p120
95-by-35+Elec.Adv p121
95-by-35+Elec.Adv p122
95-by-35+Elec.Adv p123
95-by-35+Elec.Adv p124
95-by-35+Elec.Adv p125
95-by-35+Elec.Adv p126
95-by-35+Elec.Adv p127
95-by-35+Elec.Adv p128
95-by-35+Elec.Adv p129
95-by-35+Elec.Adv p13
95-by-35+Elec.Adv p130
95-by-35+Elec.Adv p131
95-by-35+Elec.Adv p132
95-by-35+Elec.Adv p133
95-by-35+Elec.Adv p134
95-by-35+Elec.Adv p14
95-by-35+Elec.Adv p15
95-by-35+Elec.Adv p16
95-by-35+Elec.Adv p1

95-by-35.Adv p53
95-by-35.Adv p54
95-by-35.Adv p55
95-by-35.Adv p56
95-by-35.Adv p57
95-by-35.Adv p58
95-by-35.Adv p59
95-by-35.Adv p6
95-by-35.Adv p60
95-by-35.Adv p61
95-by-35.Adv p62
95-by-35.Adv p63
95-by-35.Adv p64
95-by-35.Adv p65
95-by-35.Adv p66
95-by-35.Adv p67
95-by-35.Adv p68
95-by-35.Adv p69
95-by-35.Adv p7
95-by-35.Adv p70
95-by-35.Adv p71
95-by-35.Adv p72
95-by-35.Adv p73
95-by-35.Adv p74
95-by-35.Adv p75
95-by-35.Adv p76
95-by-35.Adv p77
95-by-35.Adv p78
95-by-35.Adv p79
95-by-35.Adv p8
95-by-35.Adv p80
95-by-35.Adv p81
95-by-35.Adv p82
95-by-35.Adv p83
95-by-35.Adv p84
95-by-35.Adv p85
95-by-35.Adv p86
95-by-35.Adv p87
95-by-35.Adv p88
95-by-35.Adv p89
95-by-35.Adv p9
95-by-35.Adv p90
95-by-35.Adv p91
95-by-35.Adv p92
95-by-35.Adv p93
95-by-35.Adv p94
95-by-35.Adv p95
95-by-35.Adv p96
95-by-35.Adv p97
95-by-35.Adv p98
95-by-35.Adv p99
95-by-35.Adv+DR p1
95-by-35.Adv+DR p10
95-by-35.Adv+DR p100
95-by-35.Adv+DR p101
95-by-35.Adv+DR p102
95-by-35.Adv+DR p103
95-by-35.Adv+D

Reference.Adv+DR p53
Reference.Adv+DR p54
Reference.Adv+DR p55
Reference.Adv+DR p56
Reference.Adv+DR p57
Reference.Adv+DR p58
Reference.Adv+DR p59
Reference.Adv+DR p6
Reference.Adv+DR p60
Reference.Adv+DR p61
Reference.Adv+DR p62
Reference.Adv+DR p63
Reference.Adv+DR p64
Reference.Adv+DR p65
Reference.Adv+DR p66
Reference.Adv+DR p67
Reference.Adv+DR p68
Reference.Adv+DR p69
Reference.Adv+DR p7
Reference.Adv+DR p70
Reference.Adv+DR p71
Reference.Adv+DR p72
Reference.Adv+DR p73
Reference.Adv+DR p74
Reference.Adv+DR p75
Reference.Adv+DR p76
Reference.Adv+DR p77
Reference.Adv+DR p78
Reference.Adv+DR p79
Reference.Adv+DR p8
Reference.Adv+DR p80
Reference.Adv+DR p81
Reference.Adv+DR p82
Reference.Adv+DR p83
Reference.Adv+DR p84
Reference.Adv+DR p85
Reference.Adv+DR p86
Reference.Adv+DR p87
Reference.Adv+DR p88
Reference.Adv+DR p89
Reference.Adv+DR p9
Reference.Adv+DR p90
Reference.Adv+DR p91
Reference.Adv+DR p92
Reference.Adv+DR p93
Reference.Adv+DR p94
Reference.Adv+DR p95
Reference.Adv+DR 

In [13]:
PCAs = rawdf.index.get_level_values('PCA').unique()
scenarios = rawdf.index.get_level_values('scenario').unique()
years = rawdf.index.get_level_values('year').unique()

In [14]:
rawdf.loc[(scenarios[1])].head()

,,Capacity (GW)
year,PCA,
2010,p1,0.000015
2012,p1,0.009394
2014,p1,0.022386
2016,p1,0.055574
2018,p1,0.090955


In [15]:
rawdf.loc[scenarios[1]].head()

,,Capacity (GW)
year,PCA,
2010,p1,0.000015
2012,p1,0.009394
2014,p1,0.022386
2016,p1,0.055574
2018,p1,0.090955


In [16]:
rawdf.loc[[scenarios[1]]].head()

Capacity (GW)
scenario      year PCA               
Reference.Adv 2010 p1        0.000015
              2012 p1        0.009394
              2014 p1        0.022386
              2016 p1        0.055574
              2018 p1        0.090955